# About Unit 2

Welcome back to the Marquette University AIM time series analysis curriculum! After getting a foundational view of what makes up a time series in the last unit, we will now begin to play with some data. In this unit you will learn about preprocessing the values of a time series through several projects on handling missing values, transforming, and normalizing data.

# Getting Started

**Import Packages**

Run the following code to bring the necessary packages into your environment. Ensure you are running a python kernel >=3.0.0.

In [ ]:
!pip install PyKrige

import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

from sklearn.preprocessing import StandardScaler, MinMaxScaler, normalize, PowerTransformer
from pykrige.uk import UniversalKriging

# Load a Sample Dataset

For this week's data, we are going to pull stock prices from Yahoo Finance. Yahoo provides a very easy to use API, which is wrapped by the yfinance package. We will be looking at the daily closing values for Microsoft (MSFT) between 1/1/2010 and 12/31/2014. Run the following code to bring the data into your environment.

In [ ]:
msft = yf.Ticker('MSFT')
hist = msft.history(start='2010-01-01', end='2014-12-31', period='1d')['Close']

plt.plot(hist, label='MSFT')
plt.title('MSFT Close | 2010-2015')
plt.xlabel('Date')
plt.ylabel('Close Price (USD)')
plt.legend()
plt.show()

# Preprocessing a Time Series

You may be wondering what is preprocessing, and why would a statistician want to do it? Preprocessing is an encompassing term that describes the process of cleaning and making data ready for modelling. Preprocessing is a critical first step of analyzing a time series because raw data can contain many different flaws that make it incredibly difficult or even impossible to work with directly. In the last unit, we talked about removing outliers from our Sea Surface Temperature dataset. Removing outliers is one example of preprocessing, but there are many other ways that data can be dirty. We will look at how to account for data that has these problems in this unit.

# Missing Values

Time series that have missing values can be a real issue for analysts. The mathematics behind discrete time series analysis assumes that each value in the series is at a regularly spaced interval, and if holes exist these models can break down. An easy way that researchers solve this is by using methods of estimation around the missing points.

Since our data from Yahoo does not suffer from missing values, let's create random holes in our data to practice with. We will do this using the code below.

In [ ]:
holes = random.sample(range(0, len(hist)), k=500)
hist_missing_values = hist.copy()
hist_missing_values.iloc[holes] = np.NaN

hist_missing_values.plot()
plt.title('MSFT Close | Missing Data | 2010-2015')
plt.xlabel('Date')
plt.ylabel('Close Price (USD)')
plt.show()

**Interpolation**

Interpolation is the process of estimating unknown values that fall within the range of known data points. There are many ways to interpolate data, each with their own benefits and drawbacks. We will discuss methods of interpolation and when to use them, as well as compare our interpolated data to the original series. The difference between the true value and our estimate is called the interpolation error.

**Last Observation Carried Forward (LOCF)/Next Observation Carried Back (NOCB)**

These are the most basic types of interpolation, and do not require any math to use. In an LOCF interpolation the last value before the missing point is copied forward to fill the gap. Likewise, in an NOCB interpolation the first value after the missing point is used. These types of interpolation are useful when we can assume that values are unlikely to change much between periods, such as our stock data.

Using the ffill() and bfill() methods in pandas, we can easily estimate our missing values this way.

In [ ]:
locf = hist_missing_values.ffill()
nocb = hist_missing_values.bfill()

fig, axs = plt.subplots(3, 1, sharex=True, sharey=True)

axs[0].plot(hist, label='True Values')
axs[0].legend()
axs[1].plot(locf, label='LOCF')
axs[1].legend()
axs[2].plot(nocb, label='NOCB')
axs[2].legend()

plt.show()

**Linear Interpolation**

Linear interpolation is the method of interpolating values by drawing a straight line between two known points and using this to estimate the missing values. Linear interpolation can also be thought of as a weighted average. To calculate a linear interpolation formula, one can first find the slope of the line using $m = \frac{y_2 - y_1}{x_2 - x_1}$, and then solve the corresponding point-slope form of a line $y_2 - y_1 = m(x_2 - x_1)$. In Python, we can use the pandas interpolate method and set the interpolation method to 'linear'. Linear interpolation is best when linear trends exist in the data and the gaps are across small intervals.

In [ ]:
linear = hist_missing_values.interpolate(method='linear')

fig, axs = plt.subplots(2, 1, sharex=True, sharey=True)

axs[0].plot(hist, label='True Values')
axs[0].legend()
axs[1].plot(linear, label='Linear Interpolation')
axs[1].legend()

plt.show()

**Polynomial Interpolation**

Polynomial interpolation can be thought of as the 'advanced' version of linear interpolation, but the best way to describe the relationship is to say that linear interpolation is a special case of polynomial interpolation when $order = 1$. Polynomial interpolation also estimates missing values by drawing a line between known values, however this time polynomials are used to fit one function through multiple points. The two most common ways to find the polynomial used for the interpolation is through Lagrange polynomials or Newton polynomials, but that is beyond the scope of this unit. To calculate the order of the polynomial we want to use, we can set our maximum order to be $n - 1$ and test each option using an error metric. We must optimize our order since if the order is too high we will have an overfit model, while if the order is too low we will have an underfit model.

For this example, we will find our maximum order and use a loop to test all possible orders. Then, we will compare the Mean Squared Error to determine which order is best. Since we have so many points it would take a long time to check every single option. A good rule of thumb is to use $order = \frac{n}{3}$ as the maximum order. Due to the way in which the interpolate function works, pandas requires the polynomial degree to be odd, so we will increment by 2 starting from 3.

In [ ]:
n = hist_missing_values.count()
max_order = int(n/3)
if max_order % 2 == 0:
  max_order += 1

mse_dict = {}
for o in range(3, max_order, 2):
  poly = hist_missing_values.interpolate(method='polynomial', order=o)
  mse_dict[o] = ((hist - poly)**2).mean()
best_order = min(mse_dict, key=mse_dict.get)

poly = hist_missing_values.interpolate(method='polynomial', order=best_order)
fig, axs = plt.subplots(2, 1, sharex=True, sharey=True)

axs[0].plot(hist, label='True Values')
axs[0].legend()
axs[1].plot(poly, label=f'Polynomial Interpolation, Order: {best_order}')
axs[1].legend()

plt.show()

**Spline Interpolation**

Spline interpolation is another special case of polynomial interpolation where the polynomial used is a piecewise polynomial called a spline. A spline is a function composed of polynomial segments defined between specific points known as knots. The idea behind spline interpolation is to connect all knots with a smooth curve while minimizing the bending of the line that connects them. When the polynomials used are of degree 3 and higher, the spline takes on 3 special properties:

\begin{cases}
    q_i(x_i) = q_{i+1}(x_i) = y_i \\
    q'_i(x_i) = q'_{i+1}(x_i) & 1 \leq i \leq n-1\\
    q''_i(x_i) = q''_{i+1}(x_i)
\end{cases}

Property 1: Interpolation across all points

Property 2: First derivative continuity at the internal knots

Property 3: Second derivative continuity at the internal knots

Because of these three properties, the traditional approach to working with splines is to use polynomials of exactly degree 3, called cubic splines.

**Natural Cubic Splines**

Natural cubic splines are a special type of spline that in addition to the three main conditions previously dicussed, have an additional one regarding the endpoints: $q''_1(x_0) = q''_n(x_n) = 0$. By setting the second derivative of the endpoints to 0, we can ensure the spline behaves "naturally" at the edges and prevent excessive bending.

**Clamped Cubic Splines**

Clamped cubic splines are another special type of spline that in addition to the three main conditions, includes an additional two regarding the endpoints:

\begin{cases}
    q'_1(x_0)=f'(x_0) \\
    q'_n(x_n)=f'(x_n)
\end{cases}

By "clamping" the slope of the spline to that of the slope of the interpolated function at each endpoint, we are able to better control endpoint behavior and remove odd curves from forming at the ends.

**Not-a-Knot Splines**

Not-a-knot splines are a third special type of spline, that in addition to the three main conditions adds two new ones regarding the third derivatives at sequential knots:

\begin{cases}
    q'''_1(x_1)=q'''_2(x_1) \\
    q'''_{n-1}(x_{n-1})=q'''_n(x_{n-1})
\end{cases}

This means that the third derivative of each adjacent polynomial will be equal to each other at every knot. The purpose of this is to make the curve appear as smooth as possible at the interior knots by treating the first and last interior knots as if they aren't knots at all, thus preventing the introduction of any abrupt changes.

**Akima Splines**

The last type of spline we will discuss in this unit is the Akima Spline. This type of spline was created to give a good fit to curves where the second derivative changes, as it adapts well to varying second derivatives unlike standard cubic splines which only ensure a smooth second derivative. To find an Akima spline, you start by finding the slope between each knot. Then, the slopes of each spline at point $i$ are defined as the weighted average of each adjacent slope $m_{i-1}$ and $m_i$:

$s_i=\frac{|m_{i+1}-m_i|m_{i-1}+|m_{i-1}-m_{i-2}|m_i}{|m_{i+1}-m_i|+|m_{i-1}-m_{i-2}|}$

If the denominator is equal to zero, the slope at point $i$ is defined as the simple average:

$s_i=\frac{m_{i-1}+m_i}{2}$

You may have noticed that because of how these curves are computed, we are unable to calculate the splines at the endpoints because we would need a knot before and after our series begins and ends. Python allows us two ways of remedying this, by either using virtual knots or simply returning NaNs at the ends. Virtual knots are extrapolated points outside of our series that are then used to calculate the splines at each endpoint. Because they are extrapolated, end behavior can vary in accuracy.

**Spline Interpolation in Python**

Interpolation using splines is simple in Python. Similar to the linear and polynomial methods, pandas wraps SciPy's spline interpolation as well. Run the following code to compare each type of spline interpolation on our data.

In [ ]:
# Spline
spline = hist_missing_values.interpolate(method='spline', order=3)

# Natural Cubic
n_cubic = hist_missing_values.interpolate(method='cubicspline', bc_type='natural')

# Clamped Cubic
c_cubic = hist_missing_values.interpolate(method='cubicspline', bc_type='clamped')

# Not-a-Knot
not_a_knot = hist_missing_values.interpolate(method='cubicspline')

# Akima
akima = hist_missing_values.interpolate(method='akima')

fig, axs = plt.subplots(6, 1, sharex=True, sharey=True)

axs[0].plot(hist, label='True Values')
axs[0].legend()
axs[1].plot(spline, label='Cubic Spline')
axs[1].legend()
axs[2].plot(n_cubic, label='Natural Cubic')
axs[2].legend()
axs[3].plot(c_cubic, label='Clamped Cubic')
axs[3].legend()
axs[4].plot(not_a_knot, label='Not-a-Knot')
axs[4].legend()
axs[5].plot(akima, label='Akima')
axs[5].legend()

plt.show()

**Interpolation by Gaussian Process Regression (Kriging)**

Kriging is a method of interpolation based on Gaussian processes where the data points exhibit a spacial or temporal relationship. One of the major advantages of this technique is the creation of credible intervals along the interpolated range. Imagine you're measuring something like rainfall in several locations across a region. You want to estimate the rainfall at other places where you haven't measured it yet. Simple interpolation methods might give you an estimate based on distance from known points, but Kriging goes further. It doesn't just take distance into account; it also considers how the values you're measuring change over space. Kriging uses a covariance function to describe how similar points are based on how far apart they are. This means the method understands that points closer to each other influence the predicted value more than distant ones. An important thing to note, however is that because of the use of Gaussian processes, this method of interpolation assumes that the data is normally distributed. The predicted value at an unknown point is a weighted average of the known values around it. The weights aren't just based on distance; they're calculated using the covariance and are chosen to minimize the prediction error. There are many types of Kriging, but the three main versions are Simple, Ordinary, and Universal.

**Simple Kriging**

Simple kriging assumes that the mean of the data is known and constant.

**Ordinary Kriging**

Ordinary kriging assumes the mean is unknown, but constant in the range of values.

**Universal Kriging**

Universal Kriging assumes that the data has a trend or drift that changes over space, that is it assumes a general polynomial trend model.

**Interpolation by Kriging on our Stock Data**

Because we know that the mean is not constant within our time series (in the next unit we will discuss this when we talk about Stationarity), the correct model to use for our data is Universal Kriging. We can do this using the Python package PyKrige. Run the following code to interpolate our time series.

In [ ]:
X = hist_missing_values[hist_missing_values.notna()].index.astype(np.int64) // 10**9
Y = np.zeros(X.shape)
Z = hist_missing_values[hist_missing_values.notna()].values

missing_X = hist_missing_values[hist_missing_values.isna()].index.astype(np.int64) // 10**9
missing_Y = np.zeros(missing_X.shape)

uk = UniversalKriging(X, Y, Z, variogram_model='exponential')
interpolated_prices, _ = uk.execute('points', missing_X, missing_Y)

hist_missing_values.loc[hist_missing_values.isna()] = interpolated_prices

fig, axs = plt.subplots(2, 1, sharex=True, sharey=True)

axs[0].plot(hist, label='True Values')
axs[0].legend()
axs[1].plot(hist_missing_values, label='Kriging Interpolation')
axs[1].legend()
plt.show()

**The Semivariogram Model**

You may have noticed a parameter in the kriging class that we have not covered: the variogram model. In this particular case the input refers to the semivariogram (half of the variogram). The semivariogram model chosen to use when interpolating data by Kriging is incredibly important. This describes the spacial correlation between points based on their distance. A semivariogram model usually has three parameters: the nugget, sill, and range.

**The Nugget**

The nugget represents the spatial variation at very small scales. It is the semivariance when points are theoretically in the same place.

**The Sill**

This is the value at which the semivariance levels off. It represents that beyond a certain distance, points are no longer spacially correlated and semivariance becomes constant.

**The Range**

The range is the distance at which the semivariance reaches the sill. Beyond this range, points are considered uncorrelated.

**Different Semivariogram Models**

Just like there are many different types of Kriging, there are also many different types of semivariogram models. The ones readily available to us in Python are the linear, power, spherical, exponential, and gaussian. As you read through the characteristics of each, try to determine why I chose to use the exponential semivariogram for our stock data.

**Linear Semivariogram**

With a linear semivariogram semivariance increases linearly with distance. It's a simple model and as such may not always fit complex spatial structures well. This can be represented with the equation: $\gamma(h)=m*h$, where $m$ is the slope and $h$ is the lag distance.

**Power Semivariogram**

A power semivariogram is used when the semivariance doesn't reach a sill and keeps increasing with distance, meaning the spatial correlation persists over long distances. An equation that describes this can be written as: $\gamma(h)=c_0+m*b^h$, where $m$ and $b$ are constants.

**Spherical Semivariogram**

The spherical semivariogram describes when semivariance increases with distance up to a certain range, beyond which it levels off (i.e. points beyond the range have no correlation). Its formula could be written as:

\begin{cases}
    0 & h = 0 \\
    \gamma(h) = c_0 + (c*(\frac{3}{2}*\frac{h}{a} - \frac{1}{2}*(\frac{h}{a})^3)) & 0 < h \leq a \\
    \gamma(h) = c_0 + c & h > a
\end{cases}

Where $a$ is the range, $c$ is the sill, and $h$ is the lag.

**Exponential Semivariogram**

The exponential model is similar to the spherical, however semivariance increases rapidly then levels off more slowly compared to the spherical model. This is often used when the correlation decreases gradually. This equation can be written as $\gamma(h)=c_0+c*(1-e^{\frac{-h}{a}})$. Here $a$ is related to the range, $c_0$ is the nugget and $c$ is the sill.

**Gaussian Semivariogram**

Under the gaussian model, semivariance follows an S-shaped curve that begins and tapers off slowly, but rapidly increases near the middle. Its formula can be represented by the piecewise function:

\begin{cases}
    \gamma(h) = 0 & h=0 \\
    \gamma(h) = C_0 + C(1-e^{\frac{-h^2}{a^2}}) & h>0
\end{cases}


# Transforming Data

Once we have our data loaded and cleaned, there may still be other transformations we need to do to it. What happens if the data is not normally distributed? Does it matter if the values are really large? In this section we will be covering when and how to transform a complete time series so that models can understand it well.

**Normalization**

Normalization is the process of scaling a range of values to be (typically) between [0,1] or [-1,1]. Many models perform better, particularly neural networks, if the values are low. Normalization does not however change the distribution; in other words, the standard deviation and mean stay the same proportionally.

**Standardization**

Standardization is the process of transforming a range to values to have a mean of 0 and a standard deviation of 1. In other words, standardization converts a time series to follow a standard normal distribution. Many models require or perform better on data that is distributed this way. While we may not always be working with a series that follows the standard normal distribution, we can use this method to rescale the values. Note: standardization does not convert non-normal data to fit a normal distribution, it takes normally distributed data and standardizes it to the standard normal.

**Important Note**

Many textbooks, research papers, etc. will use the terms normalization and standardization interchangibly. While this is not true, it has become so widespread in statistics that you will definitely run into it. Do not be confused, the authors are probably just wrong. The best way to think about the relation is that standardization is a special case under the normalization umbrella.

**Z-Score Standardization**

As previously discussed, standardization is the transformation of a dataset to have a mean of 0 and a standard deviation of 1. As a result, the transformed values become dimensionless, that is it has no unit. The formula for standardization can be described as: $Z = \frac{X - \mu}{\sigma}$. You probably recognize this from any basic stats class where you worked with Z-scores.

Standardizing in Python is very simple using the scikit-learn package. Run the code below to standardize to z-scores. Additionally, check out the histogram that is generated alongside the standardized values, and notice how the data does not follow a normal distribution at all. This is a perfect example of when not to use standardization in real life.

In [ ]:
scaler = StandardScaler()
hist_standardized = scaler.fit_transform(hist.to_numpy().reshape(-1, 1))

fig, axs = plt.subplots(1,2)
axs[0].plot(hist_standardized)
axs[1].hist(hist_standardized)
plt.show()

**Methods of Normalization**

While there are many different methods of normalization, here are the ones that we will be covering in this unit:

*   Min-Max Scaling
*   Mean Absolute Normalization
*   Decimal Scaling Normalization
*   L1 Normalization
*   L2 Normalization

**Min-Max Scaling**

Min-max scaling is one of, if not the most common methods of normalization. This transformation ensures that the minimum value in the dataset is transformed to 0, the maximum value to 1, and all other values fall proportionally between 0 and 1. Its equation is represented by $X_{\text{scaled}} = \frac{X - X_{\text{min}}}{X_{\text{max}} - X_{\text{min}}}$.

**Mean Absolute Normalization**

Mean absolute normalization is a technique used to scale the values of a dataset so that they are centered around zero with a consistent range. It works by subtracting the mean of the data from each data point and dividing by the absolute mean deviation. This method can be useful when you want to rescale data while retaining information about how far data points are from the mean, but without the influence of outliers that can affect other techniques like standardization. It's formula is described by $X_{\text{normalized}} = \frac{X - \mu}{\frac{1}{n} \sum_{i=1}^{n} |X_i - \mu|}$.

**Decimal Scaling Normalization**

Decimal scaling normalization is a method of normalizing data by moving the decimal point of values. The purpose is to scale the values so that they fall within a particular range, typically between -1 and 1 (but not always). Decimal scaling is a simple method often used when the range of values is relatively small and easy to adjust by moving the decimal point. It's equation is $X_{\text{normalized}} = \frac{X}{10^j}$, where $j$ is the smallest integer such that $\frac{X}{10^j}$ brings the maximum absolute value of $X$ to be less than 1.

**L1 Normalization**

L1 normalization, also known as least absolute deviations normalization or Manhattan normalization, is a technique used to rescale the values of a dataset so that the sum of the absolute values (or norms) of the elements in a feature vector becomes 1. It is commonly applied in machine learning and data preprocessing to ensure that the magnitude of each feature contributes proportionally to a model, especially in situations where feature scaling is necessary. As an additional note, L1 normalization is particularly useful when dealing with sparse data or when you need to regularize features that should have equal weight in contributing to the model. Its formula is represented by $X_{\text{normalized}} = \frac{X}{\sum_{i=1}^{n} |x_i|}$, where $X$ is the original vector, and $x_i$ is the $i^{th}$ element of the original vector $X$.

**L2 Normalization**

L2 normalization, also known as Euclidean norm normalization, is a technique used to rescale the data such that the sum of the squares of the elements in a vector equals 1. It ensures that the magnitude of the vector is constrained while preserving the direction. L2 normalization is frequently used in machine learning, particularly for algorithms that rely on distances or dot products (e.g., k-NN, SVM, and linear regression). Its equation is $X_{\text{normalized}} = \frac{X}{\sqrt{\sum_{i=1}^{n} x_i^2}}$, where once again $X$ and $X_i$ relate to the original vector and its individual elements, respectively.

**Normalization in Python**

Each of these methods is very easy to do in Python by leveraging the scikit-learn package as well as hard coding the formulas. Run the code below to perform each type of normalization on our stock data and then graph the results.

In [ ]:
# Min-Max Scaling
scaler = MinMaxScaler()
hist_min_max_scaled = scaler.fit_transform(hist.to_numpy().reshape(-1, 1))

# Mean Absolute Normalization
mean = np.mean(hist, axis=0)
mean_abs_dev = np.mean(np.abs(hist - mean), axis=0)
hist_mean_abs_normalized = ((hist - mean) / mean_abs_dev).to_numpy()

# Decimal Scaling Normalization
j = np.ceil(np.log10(np.max(np.abs(hist), axis=0)))
hist_decimal_scaled = (hist / (10 ** j)).to_numpy()

# L1 Normalization
hist_l1_normalized = normalize(hist.to_numpy().reshape(-1,1), norm='l1', axis=0)

# L2 Normalization
hist_l2_normalized = normalize(hist.to_numpy().reshape(-1,1), norm='l2', axis=0)

# Plotting
fig, axs = plt.subplots(5, 1)
axs[0].plot(hist_min_max_scaled, label='Min-Max')
axs[0].legend()
axs[1].plot(hist_mean_abs_normalized, label='Mean Absolute')
axs[1].legend()
axs[2].plot(hist_decimal_scaled, label='Decimal Scaling')
axs[2].legend()
axs[3].plot(hist_l1_normalized, label='L1')
axs[3].legend()
axs[4].plot(hist_l2_normalized, label='L2')
axs[4].legend()
plt.show()


**Power Transformations**

Remember how in our standardization example we determined that we should not convert our data to Z-scores because it was not normally distributed? What if there was a way to fix this problem? Power transformations are a set of  transformations that aim to make a dataset more normally distributed or to stabilize variance across different levels of data. These transformations are particularly useful when the data exhibits skewness or heteroscedasticity. By applying a power transformation, you can often reduce the skew, bring the data closer to normality, and improve the performance of certain machine learning models or statistical tests that assume normally distributed data. The power transformations we will cover in this unit are:

*    Log Transformation
*    Square Root Transformation
*    Inverse Transformation
*    Box-Cox Transformation
*    Yeo-Johnson Transformation

**Log Transformation**

The log transformation is quite literally the application of a logarithmic function to the dataset. It is commonly used when data has a right-skewed distribution, meaning there are a few large values that dominate the distribution. By applying the log transformation, the spread of large values is compressed, while the relative differences between smaller values are retained or even expanded. Its formula is represented as $X_{\text{transformed}} = \log(X)$.

It is important to note that there are several times when it is inappropriate to use this method:

*    Data with negative values
*    Symmetric or left-skewed data
*    Data with zeros

**Square Root Transformation**

Similar to the log transformation, the square root transformation is a technique used to reduce right-skewness in a dataset and make it more symmetric and closer to a normal distribution. The square root transformation is especially effective when dealing with moderately skewed data. By applying the square root function to the data, large values are compressed, while smaller values are spread out more evenly. Its formula is of course $X_{\text{transformed}} = \sqrt{X}$, and transformation of negative values is not possible.

**Inverse Transformation**

The inverse transformation is a mathematical transformation where each value in a dataset is replaced by its reciprocal. This transformation is typically used to reduce the impact of large values and spread out smaller values. Inverse transformations can be particularly useful in handling left-skewed data, as it has the effect of reversing the skew by compressing large values and expanding small values. Its formula can be represented as $X_{\text{transformed}} = X^{-1} = \frac{1}{X}$.

**Box-Cox Transformation**

The Box-Cox transformation is actually a family of power transformations that are defined by a piecewise function. The Box-Cox transformation is parameterized by a parameter $λ$, which controls the nature and strength of the transformation. Its equation set is defined as:

\begin{cases}
\frac{X^{\lambda} - 1}{\lambda}, & \text{if} \, \lambda \neq 0 \\
\log(X), & \text{if} \, \lambda = 0
\end{cases}

Notice how the Box-Cox transform is equivalent to the square root transform when $λ = 0.5$, and is equivalent to the log transform when $λ = 0$. Likewise when $λ>0$ a power transformation is applied, compressing larger values and spreading out smaller ones; but when $λ < 0$ the data is inverted, reversing the direction of skewness and stretching small values. A $λ$ of 1 does not change the data at all.

**Yeo-Johnson Transformation**

The Yeo-Johnson Transformation is a modified version of the Box-Cox transformation that can handle both positive negative values, making it a more flexible alternative. Its equation is very closely related, however it introduces some new methods to be applied:

\begin{cases}
\frac{((X + 1)^{\lambda} - 1)}{\lambda}, & \text{if} \, X \geq 0 \, \text{and} \, \lambda \neq 0 \\
\log(X + 1), & \text{if} \, X \geq 0 \, \text{and} \, \lambda = 0 \\
-\frac{((-X + 1)^{2 - \lambda} - 1)}{2 - \lambda}, & \text{if} \, X < 0 \, \text{and} \, \lambda \neq 2 \\
-\log(-X + 1), & \text{if} \, X < 0 \, \text{and} \, \lambda = 2
\end{cases}

**Power Transformations in Python**

Each of these methods is very simple to apply in Python using the scikit-learn and numpy packages just as before. Run the code below to apply the transformations to our data and graph the results. We will be looking at the histograms this time to see how much each example is able to change our data.



In [ ]:
# Log Transformation
log_transformed = np.log(hist.values)

# Square Root Transformation
sqrt_transformed = np.sqrt(hist.values)

# Inverse Transformation
inv_transformed = 1 / hist.values

# Box-Cox Transformation
pt_bc = PowerTransformer(method='box-cox')
boxcox_transformed = pt_bc.fit_transform(hist.values.reshape(-1, 1))

# Yeo-Johnson Transformation (handles both positive and negative values)
pt_yj = PowerTransformer(method='yeo-johnson')
yeojohnson_transformed = pt_yj.fit_transform(hist.values.reshape(-1, 1))

fig, axs = plt.subplots(2,3, figsize=(7.5,5))

axs[0][0].hist(log_transformed, label='Log')
axs[0][0].legend()

axs[0][1].hist(sqrt_transformed, label='Square Root')
axs[0][1].legend()

axs[0][2].hist(inv_transformed, label='Inverse')
axs[0][2].legend()

axs[1][0].hist(boxcox_transformed, label='Box-Cox')
axs[1][0].legend()

axs[1][1].hist(yeojohnson_transformed, label='Yeo-Johnson')
axs[1][1].legend()

axs[1][2].hist(hist.values, label='Original')
axs[1][2].legend()

plt.show()

# Conclusion

Congratulation you survived a very long unit! This concludes Unit 2 of the AIM Time Series Analysis curriculum. After completing this, you should have a good grasp on what preprocessing data is, as well as some foundational techniques to go about it. Stay tuned for Unit 3 where we will dive deep into stationarity and unit roots within time series!